In [2]:
import requests
from urllib.parse import quote
from alive_progress import alive_it

## List all pages in category

In [3]:
r = requests.get("https://api.star-citizen.wiki/api/v2/vehicle-items/?limit=10000&locale=en_EN").json()["data"]

In [4]:
r = [component["name"] for component in r if component["grade"] is not None]

In [5]:
r

['IcePlunge',
 'QuikCool',
 'ZeroRush',
 'AbsoluteZero',
 'IceDive',
 'RapidCool',
 'Bracer',
 'Glacier',
 'Polar',
 'Tundra',
 'Arctic',
 'Avalanche',
 'Boreal',
 'Permafrost',
 'Blizzard',
 'Galinstan',
 'Mercury',
 'Tempest',
 'Cryo-Star SL',
 'Frost-Star SL',
 'Winter-Star SL',
 'Cryo-Star',
 'Frost-Star',
 'Winter-Star',
 'Cryo-Star EX',
 'Frost-Star EX',
 'Winter-Star EX',
 'Cryo-Star XL',
 'Frost-Star XL',
 'Winter-Star XL',
 'Eco-Flow',
 'Hydrocel',
 'Thermax',
 'Ultra-Flow',
 'CoolCore',
 'HydroJet',
 'Snowfall',
 'Snowpack',
 'Chill-Max',
 'Hydropulse',
 'Ice-Flush',
 'ThermalCore',
 'ArcticStorm',
 'BlastChill',
 'FlashFreeze',
 'ColdSnap',
 'FullFrost',
 'WhiteOut',
 'ColdSurge',
 'FrostBite',
 'FrostBurn',
 'Serac',
 'HeatSafe',
 'SnowBlind',
 'VaporBlock',
 'HeatSink',
 'IceBox',
 'NightFall',
 'Fridan',
 'Kelvid',
 'Tepilo',
 'Berian',
 'Endo',
 'Gelid',
 'Aufeis',
 'Graupel',
 'Taiga',
 'Draug',
 'Elsen',
 'Kragen',
 'LumaCore',
 'StarHeart',
 'SunFlare',
 'LuxCore',
 '

## Connect to DB

In [6]:
import redis

db = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)

## Add each one to DB

In [8]:
from parse import parse
for name in alive_it(r, force_tty=True):
    data = requests.get("https://api.star-citizen.wiki/api/v2/vehicle-items/" + quote(name), params={"locale": "en_EN", "include": "shops,shops.items"}).json()["data"]
    try:
        plaintext = parse(data, "component")
        #print(plaintext)
    except Exception as e:
        print(e)
        continue
    db.json().set(f"docs:{name}", redis.commands.json.path.Path.root_path(), {
        "title": name,
        "content": plaintext,
        "url": "example.com"
    })

on 200: components/radar.jinja                                                  
on 201: components/radar.jinja                                                  
on 202: components/radar.jinja                                                  
on 203: components/radar.jinja                                                  
on 204: components/radar.jinja                                                  
on 205: components/radar.jinja                                                  
on 206: components/radar.jinja                                                  
on 207: components/radar.jinja                                                  
on 208: components/radar.jinja                                                  
on 209: components/radar.jinja                                                  
on 210: components/radar.jinja                                                  
on 211: components/radar.jinja                                                  
on 212: components/radar.jin

## Do a test query

In [9]:
from redis.commands.search.query import Query
from json import loads
index = db.ft("idx:docs")

In [22]:
res = index.search(Query('@content:Size 2 Military C Power Plant'))
print(loads(res.docs[0].json)["content"])

# Ship Component: TurboDrive (Size 2 Military Grade C Power Plant)
The TurboDrive is a common military-grade power plant from A&R that slightly favors performance over durability and stealth.

## Specs
Power output: 9375

## Buyable at
Dumper's Depot, GrimHEX (39,500 aUEC)
Platinum Bay, HUR-L5 (39,500 aUEC)

